# ECE 795 - Big Data - Assignment #2
#  Crawling Data from the Internet to BigQuery


## Before you begin
**BE AWARE that you can only do a maximum of 96 DML (update, insert, delete) operations on a table per day. Be careful when you do such task in the assignment and start to work on the assignment several days before the deadline**

[New York Social Diary](http://www.newyorksocialdiary.com/) is a website providing news are reports about the social life of the elite New Yorkers. Take a look at one of [the recent post](http://www.newyorksocialdiary.com/social-diary/2019/in-the-mood). Among these news and articles, you will be able to scatch the social network of the New York's social elites.

In this project, we will utlize the website and crawl data from it to obtain some latent information in the news and articles. All the photos are carefully captioned and labelled with those who appear in the photos. This give us a perfect chance to analyze the connections between these people. We can assume that people, who appear in the same picture together, have a connection. 

For this assignment, we will assemble the social graph from photo captions, store the data in BigQuery, and perform some basic analysis on the data.

## Crawling Data

### Get list of "party pictures" from the website

Take a look at [the archive page of party pictures](http://www.newyorksocialdiary.com/party-pictures).
You will find a list of links, each of which points at a post of party pictures. On each page, 50 links are presented.

In the following, we will get the url for each party page, along with its date.

Here are some packages that you may find useful.  You are welcome to use others, if you prefer.

In [0]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

#### Download HTML of webpages
If you open a website with any browser, your browser's developer tools (usually `Cmd`-`Option`-`I` on Mac, `Ctrl`-`Shift`-`I` on others) offer helpful tools to explore the structure of the HTML page. You can also right click the page and select "Inspect"  to achieve the same purpose.

In the inspection window, you will be able to see the structures of the webpage. They are the codes in [HTML language](https://en.wikipedia.org/wiki/HTML) that tell your browser what to display .
In this project, we will extract useful information from HTML.
We recommend using Python [Requests](http://docs.python-requests.org/en/master/) to download the HTML pages, and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to process the HTML. 

Let's start by getting the [first page of "party pictures"](http://www.newyorksocialdiary.com/party-pictures).*

In [0]:
page = requests.get("http://www.newyorksocialdiary.com/party-pictures") # Use requests.get to download the page.

#### Parse HTML with BeautifulSoup
BeautifulSoup provides a parser to understand the structure of HTML. After investigate the structure of the webpage and find a pattern, you can then use BeautifulSoup's [select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) or [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) methods to get those elements.

In [0]:
soup = BeautifulSoup(page.text, "lxml")
links = soup.find_all('div', attrs={'class':'views-row'})
print(links[0])

<div class="views-row views-row-1 views-row-odd views-row-first">
<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2019/touring-and-traveling-back-in-time">Touring and traveling back in time</a></span> </span>
<span class="views-field views-field-created"> <span class="field-content">Monday, February 11, 2019</span> </span> </div>


We should be able to find 50 links from the webpage.

In [0]:
assert len(links) == 50

Let's take a look at that first link.  Figure out how to extract the URL of the link, as well as the date.  You probably want to use `datetime.strptime`.  See the [format codes for dates](https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior) for reference.

In [0]:
# Check that the title and date match what you see visually.
first_link = links[0]
first_url = first_link.find('a').attrs['href']
first_date = datetime.strptime("".join(first_link.find_all('span',{'class':'field-content'})[1].get_text()), '%A, %B %d, %Y')
print(first_url, first_date)

/party-pictures/2019/touring-and-traveling-back-in-time 2019-02-11 00:00:00


For purposes of code reuse, let's put that logic into a function.  It should take the link element and return the URL and date parsed from it.

In [0]:
def get_link_date(links, el):
    url_temp = links[el].find('a').attrs['href']
    date = datetime.strptime("".join(links[el].find_all('span',{'class' : 'field-content'})[1].contents), '%A, %B %d, %Y')
    return [url_temp, date]

For example, when we want to obtain the 50-th element of the page, we can use the following link. Be aware that the index starts from 0.

In [0]:
get_link_date(links, 49)

['/party-pictures/2018/in-the-spotlight', datetime.datetime(2018, 8, 2, 0, 0)]

You may want to check that it works as you expected.

Once that's working, let's write another function to parse all of the links on a page.  Thinking ahead, we can make it take a Requests [Response](http://docs.python-requests.org/en/master/api/#requests.Response) object and do the BeautifulSoup parsing within it.

In [0]:
def get_links(response):
    soup = BeautifulSoup(response.text, "lxml")
    links = soup.find_all('div', class_='views-row')
    page_links = []
    for i in range(len(links)):
        page_links.append(get_link_date(links, i))
    return page_links
#   return ... # A list of URL, date pairs


If we run this on the previous response, we should get 50 pairs.

In [0]:
assert len(get_links(page)) == 50

Now we should be ready to get all of the party URLs.  Click through a few of the index pages to determine how the URL changes.  Figure out a strategy to visit all of them.

HTTP requests are generally IO-bound.  This means that most of the time is spent waiting for the remote server to respond.  If you use `requests` directly, you can only wait on one response at a time.  [requests-futures](https://github.com/ross/requests-futures) lets you wait for multiple requests at a time.  You may wish to use this to speed up the downloading process.

In [0]:
import requests

url = "http://www.newyorksocialdiary.com/party-pictures"
response = requests.get(url, params={"page": ""})

In [0]:
def get_page_args(i):
    return {"url": url,
            "params": {"page": i}}

In [0]:
from requests_futures.sessions import FuturesSession
# You can use link_list.extend(others) to add the elements of others
# to link_list.

link_list = []

session = FuturesSession(max_workers=5)
futures = [session.get(**get_page_args(i)) for i in range(32)]


#link_list is capturing links for each page, creating a list of lists
# link_list = [get_links(future.result()) for future in futures]

#this captures each list one at a time and adds it to a single list, link_list.
for future in futures:
    link_list.extend(get_links(future.result()))


It the request_futures library is not avaliable in Colab, you can use the following command to install it

In [0]:
!pip install requests_futures

## Store Crawled Data into BigQuery Table

In order to store the data into BigQuery, we need to first create a table with appropriate schema in the dataset.

### Provide your credentials to the runtime

In [0]:
# Authenticate your student profile

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Create a new dataset in BigQuery to store your own data

Set the project_id as you did in assignment 1 and specify your desired dataset_id (letters, numbers and underscores allowed).

After excuting the code, you should be able to see the created dataset in [BigQuery Web UI](https://console.cloud.google.com/bigquery) under your project_id.

In [0]:
from google.cloud import bigquery
from google.api_core.exceptions import Conflict

project_id = 'bigdataproject-231517'  # Set your project_id here
client = bigquery.Client(project=project_id)
dataset_id = 'assignment2_dataset'  # Specify you dataset_id here

# Create a DatasetReference using a chosen dataset ID.
dataset_ref = client.dataset(dataset_id)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_ref)
# Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
try:
   dataset = client.create_dataset(dataset)  # API request
except Conflict:
   print('Dataset exists')
print('Dataset Created')

Dataset exists
Dataset Created


### Create a new table for the following examples

Before creating a table, you need to properly design the table scheme. In this example, we will create a table with three attributes: an integer as partyID, a string as the link, and an array of integer as the date.

In [0]:
schema = [
    bigquery.SchemaField('partyID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('partyLink', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('partyDate', 'DATE', mode='REQUIRED')
]

Then, you can create the table with GCP API.

In [0]:
table_ref = dataset_ref.table('party')
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request

You can verify whether the table is created successfully by cheking the BigQuery Console and running the following assert

In [0]:
assert table.table_id == 'party'

### Insert crawled data into the table



BigQuery only accept the format of Date as YYYY-MM-DD, so we need to convert the date into this format before inserting them into the table. We recommend to use [datetime.strftime](https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior) function for this purpose.

In [0]:
link_list = [(i, link_list[i][0], link_list[i][1].strftime("%Y-%m-%d")) for i in range(len(link_list))]

Then, we can stream the data into the table as mentioned in the [GCP documentation](https://cloud.google.com/bigquery/streaming-data-into-bigquery). You can also import the data from local files and Cloud Storage. It might take a while before all the data goes into the streaming buffer of the table.

There are [quota policies](https://cloud.google.com/bigquery/quotas#streaming_inserts) of streaming inserts into BigQuery.

In [0]:
errors = client.insert_rows(table, link_list)  # API request
assert errors == []

After the data appear in the streaming buffer, you may execute the queries as you did in assignment 1. The following example query the total number of entries in the table.

In [0]:
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `assignment2_dataset.party`''').to_dataframe().total[0]
assert row_count == len(list_link)

## Assignments
**In this assignment, you need to investigate the structure of the captions in the posts of "party pictures", design a method to extract necessary information according to the structure, insert data into the table, and design query to analyze them**

### Question 1: Design a query to get all the links in table "party" with the date before December 1, 2014 (including December 1, 2014).

In [0]:
# your code here

### Question 2: Investigate the structure of the obtained links from Question1 and design a method to crawl the image captions from the webpages.

In [0]:
# your code here

### Question 3: Create a new table in BigQuery as the following schema, and then store the obtained captions into the table appropriately.

In [0]:
schema2 = [
    bigquery.SchemaField('partyID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('imageID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('captionText', 'STRING', mode='REQUIRED')
]

In [0]:
# your code here

### Question 4: Design a method to extract names in the caption and store the obtained names into another new table.

There are various methods that can be utilized to extract names from the captions. The details are up to you.

Please find the following some hints to implement the method. You do not have to follow all of the hint to achieve a good result. It's okay that if you do not follow any of the hint. This question will be evaluated by the quality of your results.
  1. Some of the captions do not include names, instead,  they can be long narrative texts describe the photos or the events. You can implement some heuristic rules to filter out these captions. For example,
    - Names should be lists of nouns. There are various way to delilver this, for example, a straightforward but slow way is using [`nltk` library in Python to tag part of speech](http://www.nltk.org/book/ch05.html). Alternatively, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) can detect whether the word is name or not.
    - Look for common patterns in the captions with names (e.g. looking for the phrase `a friend`).
    - It is often not lists of names if the caption is very long. (You may use 250 characters as cutoff)
  2. The captions are in various forms and you will need to separate them accordingly. You may want to use `re.split`, which is more sophisticated but more powerful than `string.split`.
  3. "ra Lebenthal" might be found as a name but it is not. You might need to investigate what happens here.
  4. The same person might be referenced with the title, such as "Mayor Michael Bloomberg", or not "Michael Bloomberg". The title might change over the time as well. The titles might need to be filtered out so all these names can ultimately refer to the same person: "Michael Bloomberg."
  5. Sometimes, the couple might be written as eg. "John and Mary Smith" referring to "John Smith" and "Mary Smith". You might need to handle it.
  6. Check you output frequently and figure out the errors that come up. Then, you can fix the errors iteratively until you have a list that looks reasonable.
  7. You will probably find that the webpages have a slightly different HTML structures, as well as new captions fail your name parser. But don't worry if the parser isn't perfect -- just try to get the easy cases.

Once you feel that your algorithm is working well on these captions, parse all of the captions and extract all the names mentioned. Sort them alphabetically, by first name, and insert them into a new table with the given schema.

In this task, we neglect the fact that there can be multiple persons with the same name.

In [0]:
schema3 = [
    bigquery.SchemaField('imageID', 'INTEGER', mode='REQUIRED'),,
    bigquery.SchemaField('personName', 'STRING', mode='REQUIRED')
]

In [0]:
# your code here

### Question 5: Who is the most popular?
The easiest way to answer this question is to look at how many connections everyone has. If a person has been in the same picture with 5 identical people, he has 5 connections.
Design a query and find the top 100 people and their number of connections.

In [0]:
# your code here